In [18]:
!pip install pystac
!pip install pystac-client

In [19]:
import pystac
import pystac_client
import random
import folium
from typing import Dict, List

In [20]:
# Credentials and details for Azure APIM
APIM_STAC_URL = "https://os-eo-platform-rg-prod-apim.azure-api.net/stac"
APIM_AUTH_TOKEN = "d208843ea70a43d29416315322dc3228"
# Create AUTH Header object
auth_headers = {"ocp-apim-subscription-key": APIM_AUTH_TOKEN}

In [21]:
# Create Pystac client object
client: pystac_client.client.Client = pystac_client.Client.from_file(
    APIM_STAC_URL, headers=auth_headers
)

In [22]:
# Filter items
bbox: List[float] = [-3.911, 52.176, -3.753, 52.256]
COLLECTION = "landsat-c2-l2"
cql_filter_dict: Dict[str, str] = {
    "op": "<=",
    "args": [{"property": "eo:cloud_cover"}, 10.00],
}
items: pystac.ItemCollection = client.search(
    max_items=10, filter=cql_filter_dict, collections=[COLLECTION]
)
for item in items.item_collection():
    item: pystac.Item = item
    print(item.id)

LC09_L2SP_203023_20230905_02_T1
LC08_L2SP_204023_20230904_02_T1
LC08_L2SP_204022_20230904_02_T1


In [23]:
# plot all item bboxes on a map
first_item: pystac.Item = items.item_collection()[0]

bbox_for_map: Dict = (
    first_item.bbox[1],
    first_item.bbox[0],
    first_item.bbox[3],
    first_item.bbox[2],
)
print(bbox_for_map)

folium_map: folium.Map = folium.Map(
    tiles="OpenStreetMap", zoom_start=6, location=[bbox_for_map[0], bbox_for_map[1]]
)


def random_color() -> str:
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))


for item in items.item_collection():
    item: pystac.Item = item
    bbox: Dict = (item.bbox[1], item.bbox[0], item.bbox[3], item.bbox[2])
    # map bbox on folium map
    folium.Rectangle(
        bounds=[[bbox[0], bbox[1]], [bbox[2], bbox[3]]],
        fill=False,
        tooltip=item.id,
        fill_color=random_color(),
        popup=item.id,
    ).add_to(folium_map)

folium_map

(51.994395222207345, -3.775418454550722, 54.17540477779265, -0.15048037703013412)
